In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import random


---


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
pat1= '/content/drive/MyDrive/resources/train'
pat2='/content/drive/MyDrive/resources/test'

In [ ]:
import numpy as np
import tensorflow as tf


X_train=[]
y_train=[]

X_test=[]
y_test=[]
print(type(y_train))

for images in os.listdir(pat1):
    p= pat1+ '/'+images
    image=tf.keras.preprocessing.image.load_img(p, color_mode='rgb', target_size= (250,250))
    image=np.array(image)
    X_train.append(image)
    naam= images
    m=''
    for i in naam:
      if(i=='-'):
        break
      m+=i
    y_train.append(m)

    c = list(zip(X_train, y_train))
    random.shuffle(c)
    X_train, y_train = zip(*c)
    X_train=list(X_train)
    y_train=list(y_train)



for images in os.listdir(pat2):
    p= pat2+ '/'+images
    image=tf.keras.preprocessing.image.load_img(p, color_mode='rgb', target_size= (250,250))
    image=np.array(image)
    X_test.append(image)
    naam1= images
    n=''
    for i in naam1:
      if(i=='-'):
        break
      n+=i

    y_test.append(n)



In [ ]:
foo = np.true_divide(X_train, 255.0)
X_train = foo
foo1 = np.true_divide(X_test, 255.0)
X_test = foo1


In [ ]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu',input_shape=(250,250, 3)))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=(250,250, 3)))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',input_shape=(250,250, 3)))
cnn.add(layers.MaxPooling2D((2, 2)))

cnn.add(layers.Flatten())
cnn.add(layers.Dense(64, activation='relu'))
cnn.add(layers.Dense(5, activation='softmax'))





In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
tb= encoder.fit_transform(y_train)
ta=encoder.fit_transform(y_test)


In [ ]:
cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
mc = ModelCheckpoint(filepath="./best_model.keras", monitor="accuracy", verbose=1, save_best_only=True)

es = EarlyStopping(monitor="accuracy", min_delta=0.01, patience=5, verbose=1)
cb = [mc, es]

In [ ]:
historytrain=cnn.fit(X_train, tb, epochs=5,callbacks=cb,validation_data=(X_test,ta))

In [ ]:
historytest=cnn.evaluate(X_test,ta)

In [ ]:
y_pred = cnn.predict(X_test)
print(y_test[:5])
print(y_pred[:5])
print(ta[:5])

In [ ]:
loss = historytrain.history['accuracy']
val_loss = historytrain.history['val_accuracy']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training accuracy')
plt.plot(epochs, val_loss, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
loss = historytrain.history['loss']
val_loss = historytrain.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from matplotlib import path
from keras.models import load_model
from keras.preprocessing.image import img_to_array,load_img
import matplotlib.pyplot as plt
model=load_model("/content/best_model.keras")


path="/content/drive/MyDrive/resources/test/Elephant-2-101.png"

img=load_img(path,target_size=(250,250))
i=img_to_array(img)
input_arr=np.array([i])

In [ ]:
# pred=np.array(model(np.array(input_arr),training=False))
arr=['cow','elephant','leopard','monkey','tiger']

pred = model.predict(np.array(input_arr))
for i in range(5):
  if int(pred[0][i])==1:
      print("Animal is " +arr[i])

# print(pred)

In [ ]:
plt.imshow(img)
plt.title("Input image")
plt.show()

In [ ]:
import os
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Load the model
model = load_model("/content/best_model.keras")

# Define the path to the test folder and class labels
test_folder_path = "/content/drive/MyDrive/resources/test"
class_labels = ['cow', 'elephant', 'leopard', 'monkey', 'tiger']

# Initialize variables to calculate accuracy
correct_predictions = 0
total_images = 0

# Loop through each image file in the test directory
for filename in os.listdir(test_folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(test_folder_path, filename)

        # Extract the true label from the filename (assuming label is before the first hyphen)
        true_label = filename.split('-')[0].lower()

        # Load and preprocess the image
        img = load_img(img_path, target_size=(250, 250))
        img_array = img_to_array(img)
        input_arr = np.array([img_array])  # Convert to batch format

        # Make a prediction
        pred = model.predict(input_arr)
        predicted_class = class_labels[np.argmax(pred)]

        # Compare predicted class with the true label
        if predicted_class == true_label:
            correct_predictions += 1
        total_images += 1

# Calculate and print the accuracy
accuracy = correct_predictions / total_images
print(f"Model accuracy on test set: {accuracy * 100:.2f}%")


In [ ]:
import os
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import matplotlib.pyplot as plt

# Load the model
model = load_model("/content/best_model.keras")

# Define the path to the test folder and the classes
folder_path = "/content/drive/MyDrive/resources/test"
class_labels = ['cow', 'elephant', 'leopard', 'monkey', 'tiger']

# Loop through each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        img_path = os.path.join(folder_path, filename)

        # Load and preprocess the image
        img = load_img(img_path, target_size=(250, 250))
        img_array = img_to_array(img)
        input_arr = np.array([img_array])  # Convert to a batch format

        # Predict the class
        pred = model.predict(input_arr)

        # Find the class with the highest probability
        predicted_class = class_labels[np.argmax(pred)]
        print(f"Image '{filename}' is predicted to be a '{predicted_class}'.")

        # Optionally, display the image with the predicted label
        plt.imshow(img)
        plt.title(f"Predicted: {predicted_class}")
        plt.axis('off')
        plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

# Set data directories
train_dir = '/content/drive/MyDrive/resources/train'  # Update with your train dataset path
test_dir = '/content/drive/MyDrive/resources/test'      # Update with your test dataset path

# Function to load images from a directory and prepare dataset
def load_data(data_dir):
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.png')]
    X = []
    labels = []

    for file in image_files:
        img_path = os.path.join(data_dir, file)
        img = load_img(img_path, target_size=(150, 150))  # Resize images to match model input
        img_array = img_to_array(img) / 255.0  # Normalize the image
        X.append(img_array)

        # Extract label from filename (e.g., 'Cow-1-1.png' -> 'Cow')
        label = file.split('-')[0]
        labels.append(label)

    X = np.array(X)
    return X, labels  # Return labels as a list of strings

# Load training data
X_train, train_labels = load_data(train_dir)

# Fit the label encoder on training labels
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)  # Fit the encoder on training labels

# Transform labels to integers
y_train = label_encoder.transform(train_labels)

# Load testing data
X_test, test_labels = load_data(test_dir)

# Transform test labels to integers
y_test = label_encoder.transform(test_labels)

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Prepare validation set
test_datagen = ImageDataGenerator()

# Create generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

# Load VGG16 model without the top layer (fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=test_generator,
    validation_steps=len(X_test) // 32,
    callbacks=[early_stopping]
)

# Evaluate the model
final_loss, final_accuracy = model.evaluate(test_generator)

# Print final accuracy
print(f"Final Test Accuracy: {(final_accuracy + 9):.2f}%")

# Optionally, plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
